In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('vizualizações').getOrCreate()

23/11/02 11:20:35 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/02 11:20:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/02 11:20:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

# Relação Comentários X Likes (proporção)

# Relação Visualizações X Likes (proporção)

# Dia em que os vídeos são Públicados com alta frequência

# Total de Tags X Vísualizações 

# Tamanho dos títulos do Vídeos X Vísualizações

# Total Víews por Vídeo

# A duração do vídeo é importante para visualizações e interação (curtidas/comentários)?